# Topic Modeling with gensim
We'll try out [Latent Dirichlet Allocation (LDA)](https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) in [gensim](http://radimrehurek.com/gensim/index.html) on the [20 Newsgroups dataset](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_20newsgroups.html) with some simple preprocessing.

#### Install gensim

In [1]:
# !conda install gensim -y

##### imports

In [3]:
# gensim
from gensim import corpora, models, similarities, matutils

# sklearn
from sklearn import datasets
from sklearn.feature_extraction.text import CountVectorizer

# logging for gensim (set to INFO)
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Let's retain only a subset of the 20 categories in the original 20 Newsgroups Dataset.

In [4]:
# Set categories
categories = ['alt.atheism', 'comp.graphics', 'rec.sport.baseball', 
              'rec.motorcycles', 'sci.space', 'talk.politics.mideast']

# Download the training subset of the 20 NG dataset, with headers, footers, quotes removed
# Only keep docs from the 6 categories above
ng_train = datasets.fetch_20newsgroups(subset='train', categories=categories, 
                                      remove=('headers', 'footers', 'quotes'))

In [5]:
# Take a look at the first doc
ng_train.data[0]

'Well, the Red Sox have apparenly resigned Herm Winningham to a AAA contract.\nTed "Larry" Simmons signed him to a AAA contract then released him from\nBuffalo, allowing Lou "Curly" Gorman to circumvent the rule about not\nresigning free agents until May 1. Clearly, neither of these guys is bright\nenough to be Moe.\n\n Mike Jones | AIX High-End Development | mjones@donald.aix.kingston.ibm.com'

## Document Preprocessing
We'll need to generate a term-document matrix of word (token) counts for use in LDA.

We'll use `sklearn`'s `CountVectorizer` to generate our term-document matrix of counts. We'll make use of a few parameters to accomplish the following preprocessing of the text documents all within the `CountVectorizer`:
* `analyzer=word`: Tokenize by word
* `ngram_range=(1,2)`: Keep all 1 and 2-word grams
* `stop_words=english`: Remove all English stop words
* `token_pattern=\\b[a-z][a-z]+\\b`: Match all tokens with 2 or more (strictly) alphabet characters

In [6]:
# Create a CountVectorizer for parsing/counting words
count_vectorizer = CountVectorizer(ngram_range=(1, 2),  
                                   stop_words='english', token_pattern="\\b[a-z][a-z]+\\b")

count_vectorizer.fit(ng_train.data)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 2), preprocessor=None, stop_words='english',
                strip_accents=None, token_pattern='\\b[a-z][a-z]+\\b',
                tokenizer=None, vocabulary=None)

In [7]:
# Create the term-document matrix
# Transpose it so the terms are the rows
doc_word = count_vectorizer.transform(ng_train.data).transpose()

In [8]:
import pandas as pd

pd.DataFrame(doc_word.toarray(), count_vectorizer.get_feature_names()).head()

,0,1,2,3,4,5,6,7,8,9,...,3406,3407,3408,3409,3410,3411,3412,3413,3414,3415
aa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aa aaa,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aa albany,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aa atlanta,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
aa does,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
doc_word.shape

(272502, 3416)

##### Convert to gensim
We need to convert our sparse `scipy` matrix to a `gensim`-friendly object called a Corpus:

In [10]:
# Convert sparse matrix of counts to a gensim corpus
corpus = matutils.Sparse2Corpus(doc_word)  # converting data into another format that gensim can understand

##### Map matrix rows to words (tokens)
We need to save a mapping (dict) of row id to word (token) for later use by gensim:

In [11]:
id2word = dict((v, k) for k, v in count_vectorizer.vocabulary_.items())

In [12]:
len(id2word)

272502

## LDA
At this point we can simply plow ahead in creating an LDA model.  It requires our corpus of word counts, mapping of row ids to words, and the number of topics (3).

In [13]:
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaModel(corpus=corpus,  # we converted our data into a way gensim can understadn above
                      num_topics=3,   # how many topics are there
                      id2word=id2word, 
                      passes=5)   # telling gensim to run trough how many iterations

2019-11-20 14:37:35,775 : INFO : using symmetric alpha at 0.3333333333333333
2019-11-20 14:37:35,779 : INFO : using symmetric eta at 0.3333333333333333
2019-11-20 14:37:35,905 : INFO : using serial LDA version on this node
2019-11-20 14:37:36,083 : INFO : running online (multi-pass) LDA training, 3 topics, 5 passes over the supplied corpus of 3416 documents, updating model once every 2000 documents, evaluating perplexity every 3416 documents, iterating 50x with a convergence threshold of 0.001000
2019-11-20 14:37:36,085 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2019-11-20 14:37:36,196 : INFO : PROGRESS: pass 0, at document #2000/3416
2019-11-20 14:37:38,496 : INFO : merging changes from 2000 documents into a model of 3416 documents
2019-11-20 14:37:38,619 : INFO : topic #0 (0.333): 0.001*"space" + 0.001*"people" + 0.001*"just" + 0.001*"don" + 0.001*"jews" + 0.001*"like" + 0.001*"know" + 0.001*"ti

2019-11-20 14:37:58,458 : INFO : topic #0 (0.333): 0.002*"people" + 0.002*"jews" + 0.002*"turkish" + 0.002*"israel" + 0.001*"armenian" + 0.001*"like" + 0.001*"just" + 0.001*"israeli" + 0.001*"think" + 0.001*"armenians"
2019-11-20 14:37:58,465 : INFO : topic #1 (0.333): 0.002*"space" + 0.002*"people" + 0.002*"just" + 0.001*"said" + 0.001*"don" + 0.001*"like" + 0.001*"time" + 0.001*"know" + 0.001*"think" + 0.001*"didn"
2019-11-20 14:37:58,472 : INFO : topic #2 (0.333): 0.002*"don" + 0.002*"like" + 0.002*"know" + 0.001*"edu" + 0.001*"image" + 0.001*"just" + 0.001*"people" + 0.001*"good" + 0.001*"think" + 0.001*"time"
2019-11-20 14:37:58,478 : INFO : topic diff=0.231848, rho=0.386103
2019-11-20 14:38:00,628 : INFO : -11.573 per-word bound, 3045.9 perplexity estimate based on a held-out corpus of 1416 documents with 254128 words
2019-11-20 14:38:00,629 : INFO : PROGRESS: pass 4, at document #3416/3416
2019-11-20 14:38:01,204 : INFO : merging changes from 1416 documents into a model of 3416 

Let's take a look at what happened.  Here are the 5 most important words for each of the 3 topics we found:

In [14]:
lda.print_topics()   # it will give you the topics
                     # will return you the number of topics you specified above
    
    # HOWEVER, it is your job to infer what the topics are!!

2019-11-20 14:38:01,370 : INFO : topic #0 (0.333): 0.002*"people" + 0.002*"israel" + 0.002*"turkish" + 0.001*"jews" + 0.001*"armenian" + 0.001*"like" + 0.001*"just" + 0.001*"israeli" + 0.001*"think" + 0.001*"armenians"
2019-11-20 14:38:01,379 : INFO : topic #1 (0.333): 0.002*"space" + 0.002*"people" + 0.002*"just" + 0.002*"said" + 0.002*"don" + 0.001*"like" + 0.001*"time" + 0.001*"know" + 0.001*"didn" + 0.001*"think"
2019-11-20 14:38:01,387 : INFO : topic #2 (0.333): 0.002*"don" + 0.002*"image" + 0.002*"like" + 0.001*"know" + 0.001*"just" + 0.001*"people" + 0.001*"edu" + 0.001*"does" + 0.001*"think" + 0.001*"good"


[(0,
  '0.002*"people" + 0.002*"israel" + 0.002*"turkish" + 0.001*"jews" + 0.001*"armenian" + 0.001*"like" + 0.001*"just" + 0.001*"israeli" + 0.001*"think" + 0.001*"armenians"'),
 (1,
  '0.002*"space" + 0.002*"people" + 0.002*"just" + 0.002*"said" + 0.002*"don" + 0.001*"like" + 0.001*"time" + 0.001*"know" + 0.001*"didn" + 0.001*"think"'),
 (2,
  '0.002*"don" + 0.002*"image" + 0.002*"like" + 0.001*"know" + 0.001*"just" + 0.001*"people" + 0.001*"edu" + 0.001*"does" + 0.001*"think" + 0.001*"good"')]

#### Topic Space
If we want to map our documents to the topic space we need to actually use the LdaModel transformer that we created above, like so:

In [15]:
# Transform the docs from the word space to the topic space (like "transform" in sklearn)
lda_corpus = lda[corpus]
lda_corpus

In [16]:
# Store the documents' topic vectors in a list so we can take a peak
lda_docs = [doc for doc in lda_corpus]

Now we can take a look at the document vectors in the topic space, which are measures of the component of each document along each topic.  Thus, at most a document vector can have num_topics=3 nonzero components in the topic space, and most have far fewer.

In [17]:
# Check out the document vectors in the topic space for the first 5 documents
lda_docs[0:5]  # individual document vectors in the topic space

# For document 1 (first row), it is mostly made out of topic 2
# in document 2, it is mostly made out of topic 1 and topic 2

[[(2, 0.991012)],
 [(1, 0.7978154), (2, 0.19260629)],
 [(0, 0.010123005), (1, 0.01012627), (2, 0.9797507)],
 [(2, 0.9869936)],
 [(2, 0.99285764)]]

In [18]:
ng_train.data[0]

'Well, the Red Sox have apparenly resigned Herm Winningham to a AAA contract.\nTed "Larry" Simmons signed him to a AAA contract then released him from\nBuffalo, allowing Lou "Curly" Gorman to circumvent the rule about not\nresigning free agents until May 1. Clearly, neither of these guys is bright\nenough to be Moe.\n\n Mike Jones | AIX High-End Development | mjones@donald.aix.kingston.ibm.com'

## On your own...
- Pick a few subsets of the 20newsgroups dataset  
- Try performing LDA on this data with gensim
- Play with some of the preprocessing options and parameters for LDA, observe what happens
- See if you can use the resulting topic space to extract topic vectors
- How do your results look?
- Can you think of how you could cluster this data?

In [19]:
# Create lda model (equivalent to "fit" in sklearn)
lda = models.LdaModel(corpus=corpus,  # we converted our data into a way gensim can understadn above
                      num_topics=6,   # how many topics are there
                      id2word=id2word, 
                      passes=10)   # telling gensim to run trough how many iterations

2019-11-20 15:19:12,610 : INFO : using symmetric alpha at 0.16666666666666666
2019-11-20 15:19:12,628 : INFO : using symmetric eta at 0.16666666666666666
2019-11-20 15:19:12,718 : INFO : using serial LDA version on this node
2019-11-20 15:19:13,143 : INFO : running online (multi-pass) LDA training, 6 topics, 10 passes over the supplied corpus of 3416 documents, updating model once every 2000 documents, evaluating perplexity every 3416 documents, iterating 50x with a convergence threshold of 0.001000
2019-11-20 15:19:13,486 : INFO : PROGRESS: pass 0, at document #2000/3416
2019-11-20 15:19:17,531 : INFO : merging changes from 2000 documents into a model of 3416 documents
2019-11-20 15:19:17,819 : INFO : topic #1 (0.167): 0.002*"space" + 0.002*"like" + 0.001*"time" + 0.001*"don" + 0.001*"know" + 0.001*"just" + 0.001*"people" + 0.001*"think" + 0.001*"edu" + 0.001*"jews"
2019-11-20 15:19:17,834 : INFO : topic #3 (0.167): 0.002*"don" + 0.002*"know" + 0.002*"people" + 0.001*"just" + 0.001*"s

2019-11-20 15:19:46,746 : INFO : topic #0 (0.167): 0.002*"image" + 0.002*"people" + 0.002*"just" + 0.002*"like" + 0.002*"don" + 0.001*"think" + 0.001*"new" + 0.001*"know" + 0.001*"time" + 0.001*"use"
2019-11-20 15:19:46,760 : INFO : topic diff=0.406556, rho=0.460874
2019-11-20 15:19:46,933 : INFO : PROGRESS: pass 3, at document #2000/3416
2019-11-20 15:19:48,903 : INFO : merging changes from 2000 documents into a model of 3416 documents
2019-11-20 15:19:49,111 : INFO : topic #4 (0.167): 0.003*"turkish" + 0.003*"armenian" + 0.002*"jews" + 0.002*"people" + 0.002*"armenians" + 0.001*"turkey" + 0.001*"genocide" + 0.001*"israel" + 0.001*"russian" + 0.001*"turks"
2019-11-20 15:19:49,119 : INFO : topic #0 (0.167): 0.002*"image" + 0.002*"people" + 0.002*"just" + 0.002*"like" + 0.002*"don" + 0.001*"think" + 0.001*"know" + 0.001*"time" + 0.001*"new" + 0.001*"need"
2019-11-20 15:19:49,129 : INFO : topic #2 (0.167): 0.002*"don" + 0.002*"like" + 0.002*"think" + 0.002*"just" + 0.002*"know" + 0.001*"

2019-11-20 15:20:12,612 : INFO : topic diff=0.143997, rho=0.360188
2019-11-20 15:20:12,770 : INFO : PROGRESS: pass 6, at document #2000/3416
2019-11-20 15:20:14,528 : INFO : merging changes from 2000 documents into a model of 3416 documents
2019-11-20 15:20:14,771 : INFO : topic #0 (0.167): 0.002*"people" + 0.002*"just" + 0.002*"image" + 0.002*"like" + 0.001*"don" + 0.001*"think" + 0.001*"know" + 0.001*"time" + 0.001*"new" + 0.001*"greek"
2019-11-20 15:20:14,781 : INFO : topic #3 (0.167): 0.003*"people" + 0.003*"don" + 0.002*"know" + 0.002*"said" + 0.002*"just" + 0.002*"god" + 0.002*"say" + 0.002*"like" + 0.002*"didn" + 0.002*"think"
2019-11-20 15:20:14,790 : INFO : topic #2 (0.167): 0.002*"don" + 0.002*"like" + 0.002*"jpeg" + 0.002*"think" + 0.002*"just" + 0.002*"know" + 0.001*"good" + 0.001*"does" + 0.001*"people" + 0.001*"use"
2019-11-20 15:20:14,801 : INFO : topic #1 (0.167): 0.005*"space" + 0.002*"nasa" + 0.002*"edu" + 0.002*"data" + 0.001*"like" + 0.001*"time" + 0.001*"launch" + 

2019-11-20 15:20:36,928 : INFO : PROGRESS: pass 9, at document #2000/3416
2019-11-20 15:20:38,635 : INFO : merging changes from 2000 documents into a model of 3416 documents
2019-11-20 15:20:38,888 : INFO : topic #3 (0.167): 0.003*"people" + 0.003*"don" + 0.002*"know" + 0.002*"said" + 0.002*"just" + 0.002*"god" + 0.002*"say" + 0.002*"like" + 0.002*"didn" + 0.002*"think"
2019-11-20 15:20:38,901 : INFO : topic #1 (0.167): 0.005*"space" + 0.002*"nasa" + 0.002*"edu" + 0.002*"data" + 0.001*"like" + 0.001*"time" + 0.001*"launch" + 0.001*"available" + 0.001*"graphics" + 0.001*"ftp"
2019-11-20 15:20:38,908 : INFO : topic #4 (0.167): 0.003*"armenian" + 0.003*"turkish" + 0.002*"jews" + 0.002*"armenians" + 0.002*"people" + 0.001*"turkey" + 0.001*"genocide" + 0.001*"israel" + 0.001*"turks" + 0.001*"history"
2019-11-20 15:20:38,918 : INFO : topic #5 (0.167): 0.001*"people" + 0.001*"armenian" + 0.001*"said" + 0.001*"like" + 0.001*"adl" + 0.001*"information" + 0.001*"just" + 0.001*"question" + 0.001*

In [20]:
lda.print_topics()   # it will give you the topics
                     # will return you the number of topics you specified above
    
    # HOWEVER, it is your job to infer what the topics are!!

2019-11-20 15:20:44,748 : INFO : topic #0 (0.167): 0.002*"image" + 0.002*"people" + 0.002*"like" + 0.002*"just" + 0.001*"don" + 0.001*"think" + 0.001*"know" + 0.001*"time" + 0.001*"new" + 0.001*"use"
2019-11-20 15:20:44,758 : INFO : topic #1 (0.167): 0.005*"space" + 0.002*"nasa" + 0.002*"data" + 0.001*"launch" + 0.001*"like" + 0.001*"edu" + 0.001*"time" + 0.001*"satellite" + 0.001*"available" + 0.001*"israel"
2019-11-20 15:20:44,770 : INFO : topic #2 (0.167): 0.003*"jpeg" + 0.002*"don" + 0.002*"like" + 0.002*"think" + 0.002*"just" + 0.001*"know" + 0.001*"does" + 0.001*"good" + 0.001*"image" + 0.001*"people"
2019-11-20 15:20:44,781 : INFO : topic #3 (0.167): 0.003*"people" + 0.003*"don" + 0.002*"know" + 0.002*"just" + 0.002*"said" + 0.002*"god" + 0.002*"say" + 0.002*"like" + 0.002*"didn" + 0.002*"think"
2019-11-20 15:20:44,788 : INFO : topic #4 (0.167): 0.003*"armenian" + 0.003*"turkish" + 0.002*"armenians" + 0.002*"people" + 0.002*"jews" + 0.001*"genocide" + 0.001*"turkey" + 0.001*"isr

[(0,
  '0.002*"image" + 0.002*"people" + 0.002*"like" + 0.002*"just" + 0.001*"don" + 0.001*"think" + 0.001*"know" + 0.001*"time" + 0.001*"new" + 0.001*"use"'),
 (1,
  '0.005*"space" + 0.002*"nasa" + 0.002*"data" + 0.001*"launch" + 0.001*"like" + 0.001*"edu" + 0.001*"time" + 0.001*"satellite" + 0.001*"available" + 0.001*"israel"'),
 (2,
  '0.003*"jpeg" + 0.002*"don" + 0.002*"like" + 0.002*"think" + 0.002*"just" + 0.001*"know" + 0.001*"does" + 0.001*"good" + 0.001*"image" + 0.001*"people"'),
 (3,
  '0.003*"people" + 0.003*"don" + 0.002*"know" + 0.002*"just" + 0.002*"said" + 0.002*"god" + 0.002*"say" + 0.002*"like" + 0.002*"didn" + 0.002*"think"'),
 (4,
  '0.003*"armenian" + 0.003*"turkish" + 0.002*"armenians" + 0.002*"people" + 0.002*"jews" + 0.001*"genocide" + 0.001*"turkey" + 0.001*"israel" + 0.001*"turks" + 0.001*"history"'),
 (5,
  '0.001*"people" + 0.001*"said" + 0.001*"adl" + 0.001*"armenian" + 0.001*"bullock" + 0.001*"like" + 0.001*"information" + 0.001*"police" + 0.001*"san" + 0.